<a href="https://colab.research.google.com/github/ajtamayoh/NLP-CIC-WFU-Contribution-to-DisTEMIST-shared-task-2022/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP-CIC-WFU contribution to DisTEMIST 2022 sub-track 1 entities

Here you are the source code for the paper:

### mBERT and Simple Post-Processing: A Baseline for Disease Mention Detection in Spanish

Authors:

Antonio Tamayo (ajtamayo2019@ipn.cic.mx, ajtamayoh@gmail.com)

Diego A. Burgos (burgosda@wfu.edu)

Alexander Gelbulkh (gelbukh@gelbukh.com)

For bugs or questions related to the code, do not hesitate to contact us (Antonio Tamayo: ajtamayoh@gmail.com)

If you use this code please cite our work:



# Requirements

To run this code you need to download the dataset (two folders: training and test_background) at: https://drive.google.com/drive/folders/1qJQQHdEkm6YMN-1KRUPfBx8lFQKUZK5Z?usp=sharing

Then, you must create a folder called "Datasets" in the root of your Google Drive and load there both folders previously downloaded.

## About the infrastructure

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

## Connecting to Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Exploring & Preprocessing Data

In [ ]:
import pandas as pd
import numpy as np
import spacy

In [ ]:
distemist_track_1_training = pd.read_csv("/content/drive/MyDrive/Datasets/training/subtrack1_entities/distemist_subtrack1_training_mentions.tsv", delimiter="\t")
distemist_track_1_training.head()

,filename,mark,label,off0,off1,span
0,es-S0210-56912007000900007-3,T1,ENFERMEDAD,164,166,DM
1,es-S0210-56912007000900007-3,T2,ENFERMEDAD,362,376,deshidratación
2,es-S0210-56912007000900007-3,T3,ENFERMEDAD,575,590,hiperamilasemia
3,es-S0210-56912007000900007-3,T4,ENFERMEDAD,715,733,pancreatitis aguda
4,es-S0210-56912007000900007-3,T5,ENFERMEDAD,1402,1459,formación polipoidea sésil situada junto al es...


In [ ]:
list_off0 = list(distemist_track_1_training['off0'])
list_off1 = list(distemist_track_1_training['off1'])

In [ ]:
text_files_path = "/content/drive/MyDrive/Datasets/training/text_files"

In [ ]:
f = open(text_files_path + "/" + distemist_track_1_training.iloc[1,0] + ".txt", "r", encoding="UTF-8")
for l in f:
  print(l)

Mujer de 74 años que ingresó en el hospital por obnubilación y anuria tras presentar durante 5 días dolor abdominal y vómitos. Entre sus antecedentes destacaba una DM tratada con metformina (850 mg/8 h) y glibenclamida. La presión arterial era de 105/60 mmHg, la frecuencia cardíaca de 155 latidos/minuto y la temperatura de 36,7º C. En la exploración destacaba deshidratación intensa, desorientación, respiración de Kussmaul, dolor abdominal con peristaltismo débil y ausencia de defensa abdominal. La tabla 2 muestra los principales datos analíticos, destacando además una hiperamilasemia de 2.605 U/l. Una tomografía computarizada abdominal descubrió un aumento del tamaño de la cabeza del páncreas sugerente de pancreatitis aguda. Tras suspender la administración de metformina, la paciente fue intubada orotraquealmente y conectada a un respirador mecánico. También recibió tratamiento a base de fluidos con suplementos de potasio, noradrenalina, bicarbonato, insulina, amiodarona, imipenen y fu

In [ ]:
#Clinical cases
HCs = {}
for fname in distemist_track_1_training["filename"]:
  with open(text_files_path + "/" + fname + ".txt", "r", encoding="UTF-8") as f:
    HCs.update({fname: f.read()})

In [ ]:
len(HCs)

750

In [ ]:
#This code cell is optional. Uncomment it if you want to create a small development partition
'''
dev_files = ["es-S1130-14732006000500005-1", "es-S1130-14732005000200003-1", "es-S1130-05582014000200008-1", "es-S0365-66912010000600004-1", "es-S0212-71992004000600005-1", "es-S0211-57352011000100008-1", "es-S0210-48062009000600016-1", "es-S0004-06142009000200008-1", "es-S0004-06142007000600016-2"]
#Se descomenta para tener un dataset de development y observar las métricas con el script de la competencia
for f in dev_files:
  HCs.pop(f)
'''

In [ ]:
len(HCs)

750

In [ ]:
#Diseases
ENF = {}
enfermedades = []
fn = distemist_track_1_training["filename"][0]
for fname, enf in zip(distemist_track_1_training["filename"], distemist_track_1_training["span"]):
    if fname!=fn:
      enfermedades = []
    enfermedades.append(enf)
    ENF.update({fname: enfermedades})
    fn = fname

In [ ]:
len(ENF)

750

In [ ]:
#This code cell is optional. Uncomment it if you want to create a small development partition. This works together with the cell above with a similar comment.
'''
#Se descomenta para tener un dataset de development y observar las métricas con el script de la competencia
for f in dev_files:
  ENF.pop(f)
'''

'\n#Se descomenta para tener un dataset de development y observar las métricas con el script de la competencia\nfor f in dev_files:\n  ENF.pop(f)\n'

In [ ]:
len(ENF)

750

In [ ]:
HCs["es-S1139-76322015000300009-1"]

'Varón de tres años y medio afecto de daño neurológico grave secundario a accidente cerebrovascular agudo (ACV) isquémico perinatal. Acude a consulta con su madre, que nos relata un llanto inusual desde hace unas semanas y la sospecha de que el niño tiene dolor en las manos. Asimismo nos comenta que el niño recibe desde hace unas semanas tratamiento con esteroides orales, pues en el seguimiento de su daño neurológico, al consultar por los síntomas del aparato locomotor, se sospechó artritis crónica idiopática con radiografía normal de manos y muñecas.\nEs hijo único de padres no consanguíneos. El padre tiende un problema óseo en las muñecas desde la infancia; recuerda haber padecido dolores en las muñecas cuando era niño y nos refiere que no tiene los huesos de las muñecas, que conserva movilidad normal y que no le supone problemas en su profesión de conductor de coches.\nLa exploración física del niño está condicionada por su daño neurológico (emite gritos, sin lenguaje de otro tipo y

In [ ]:
ENF["es-S0210-56912007000900007-3"]

['DM',
 'deshidratación',
 'hiperamilasemia',
 'pancreatitis aguda',
 'formación polipoidea sésil situada junto al esfínter anal',
 'adenoma velloso de recto',
 'adenoma']

## Tokenization using SpaCy

In [ ]:
from spacy.lang.es import Spanish
nlp = Spanish()
# Create a Tokenizer with the default settings for Spanish
# including punctuation rules and exceptions
tokenizer_spacy = nlp.tokenizer

In [ ]:
HCs_tokenized = []
for hc in HCs:
    hl = []
    tokens = tokenizer_spacy(HCs[hc])
    #tokens = HCs[hc].split(" ") #The simplest option. It was not used in our work.
    for t in tokens:
        hl.append(str(t))
    HCs_tokenized.append(hl)

In [ ]:
len(HCs_tokenized)

750

In [ ]:
#HCs_tokenized[0]

In [ ]:
Ent_tokenized = []
for enf in ENF:
    Tks = []
    for e in ENF[enf]:
      sl = []
      tokens = tokenizer_spacy(e)
      #tokens = e.split(" ")
      for t in tokens:
          sl.append(str(t))
      Tks.append(sl)
    Ent_tokenized.append(Tks)

In [ ]:
len(Ent_tokenized)

750

In [ ]:
Ent_tokenized[0]

[['DM'],
 ['deshidratación'],
 ['hiperamilasemia'],
 ['pancreatitis', 'aguda'],
 ['formación',
  'polipoidea',
  'sésil',
  'situada',
  'junto',
  'al',
  'esfínter',
  'anal'],
 ['adenoma', 'velloso', 'de', 'recto'],
 ['adenoma']]

## Tagging Data with BIO scheme

In [ ]:
def find_idx(list_to_check, item_to_find):
    indices = []
    for idx, value in enumerate(list_to_check):
        if value == item_to_find:
            indices.append(idx)
    return indices

In [ ]:
import sys
labels_tokenized = []
idx =-1
for hct, et in zip(HCs_tokenized, Ent_tokenized):
    idx+=1
    labels = []
    for i in range(len(hct)):
        #Labels: 0->'O'; 1->'B'; 2->'I'
        #labels.append('O')
        labels.append(0)

    #For Entities (Diseases|Enfermedades)
    for enf in et:
      first = True
      for e in enf:
          if first == True:
              try:
                #labels[hct.index(e)] = 'B'
                #labels[posLab] = 'B'
                indices = find_idx(hct, e)
                if len(indices) > 1:
                  for id in indices:
                      labels[id] = 1
                else:
                  labels[hct.index(e)] = 1
                
                first = False
              except:
                first = False
                if e == "sarcoma+carcinoma" or e == "carcinoma+sarcoma":
                  continue
                print(hct)
                print(et)
                print(enf)
                print(e)
                print(idx)
                sys.exit(0)
          else:
              try:
                #labels[hct.index(e)] = 'I'
                #labels[posLab] = 'I'
                indices = find_idx(hct, e)
                if len(indices) > 1:
                  for id in indices:
                      if labels[id-1] != 0:
                        labels[id] = 2
                else:
                  labels[hct.index(e)] = 2
              except:
                if e == "sarcoma+carcinoma" or e == "carcinoma+sarcoma":
                  continue
                print(hct)
                print(et)
                print(enf)
                print(e)
                print(idx)
                sys.exit(0)

    labels_tokenized.append(labels)

In [ ]:
j = 0
for i in range(len(HCs_tokenized[j])):
  print(str(HCs_tokenized[j][i]) + "\t" + str(labels_tokenized[j][i]))

Mujer	0
de	0
74	0
años	0
que	0
ingresó	0
en	0
el	0
hospital	0
por	0
obnubilación	0
y	0
anuria	0
tras	0
presentar	0
durante	0
5	0
días	0
dolor	0
abdominal	0
y	0
vómitos	0
.	0
Entre	0
sus	0
antecedentes	0
destacaba	0
una	0
DM	1
tratada	0
con	0
metformina	0
(	0
850	0
mg/8	0
h	0
)	0
y	0
glibenclamida	0
.	0
La	0
presión	0
arterial	0
era	0
de	0
105/60	0
mmHg	0
,	0
la	0
frecuencia	0
cardíaca	0
de	0
155	0
latidos	0
/	0
minuto	0
y	0
la	0
temperatura	0
de	0
36,7º	0
C.	0
En	0
la	0
exploración	0
destacaba	0
deshidratación	1
intensa	0
,	0
desorientación	0
,	0
respiración	0
de	0
Kussmaul	0
,	0
dolor	0
abdominal	0
con	0
peristaltismo	0
débil	0
y	0
ausencia	0
de	0
defensa	0
abdominal	0
.	0
La	0
tabla	0
2	0
muestra	0
los	0
principales	0
datos	0
analíticos	0
,	0
destacando	0
además	0
una	0
hiperamilasemia	1
de	2
2.605	0
U	0
/	0
l	0
.	0
Una	0
tomografía	0
computarizada	0
abdominal	0
descubrió	0
un	0
aumento	0
del	0
tamaño	0
de	0
la	0
cabeza	0
del	0
páncreas	0
sugerente	0
de	0
pancreatitis	1
aguda	2
.	0
T

## Validating tokenization and alignment with the BIO tags.




In [ ]:
flag = 0
for st, lt in zip(HCs_tokenized, labels_tokenized):
    if len(st) != len(lt):
        print(st)
        print(lt)
        flag = 1
if flag==0:
    print("Everything is aligned!")

Everything is aligned!


## Sentence tokenization is an alternative but we finally used the whole clinical cases as samples.

In [ ]:
sent_tokenized = []
label_sent_tokenized = []
for ht, lht in zip(HCs_tokenized, labels_tokenized):
  st = []; lbst = []
  for h, l in zip(ht,lht):
    if h != ".":
      st.append(h)
      lbst.append(l)
    else:
      st.append(".")
      lbst.append(0)
      sent_tokenized.append(st)
      label_sent_tokenized.append(lbst)
      st = []; lbst = []

In [ ]:
len(sent_tokenized)

11795

In [ ]:
sent_tokenized[0]

['Mujer',
 'de',
 '74',
 'años',
 'que',
 'ingresó',
 'en',
 'el',
 'hospital',
 'por',
 'obnubilación',
 'y',
 'anuria',
 'tras',
 'presentar',
 'durante',
 '5',
 'días',
 'dolor',
 'abdominal',
 'y',
 'vómitos',
 '.']

In [ ]:
len(label_sent_tokenized)

11795

In [ ]:
label_sent_tokenized[0]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# Disease mentions identification as a Token classification problem

## Install the Transformers and Datasets libraries to run this notebook.

In [ ]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


## Hugging Face Authentication

If you want to save your own model and make it available online we strongly recommend signing up at: https://huggingface.co/

You will need to setup git, adapt your email and name in the following cell.

In [ ]:
!git config --global user.email "your_email"
!git config --global user.name "your_name"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Building the Dataset

In [ ]:
dic = {"tokens": HCs_tokenized, "ner_tags": labels_tokenized} #For the whole clinical case. We used this option for our paper.
#dic = {"tokens": sent_tokenized, "ner_tags": label_sent_tokenized} #Use this option if you want to check the model performance with sentences tokenized by ". " but the whole clinical cases.

In [ ]:
from datasets import Dataset, DatasetDict
dataset = Dataset.from_dict(dic)

In [ ]:
dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 750
})

In [ ]:
#For training, validation, and test partitions
"""
#Train, val, test partitions
train_test = dataset.train_test_split()
test_val = train_test['test'].train_test_split()
raw_datasets = DatasetDict({
    'train': train_test['train'],
    'validation': test_val['train'],
    'test': test_val['test']
    })
"""

#Just for training and validation partitions
train_test = dataset.train_test_split()
raw_datasets = DatasetDict({
    'train': train_test['train'],
    'validation': train_test['test']
    })


In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 562
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 188
    })
})

In [ ]:
raw_datasets["train"][0]["ner_tags"]
#raw_datasets["train"][0]["pos_tags"]
#raw_datasets["train"][0]["chunk_tags"]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 0,
 1,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
raw_datasets['train']

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 562
})

In [ ]:
label_names = ['O','B','I']
label_names

['O', 'B', 'I']

In [ ]:
words = raw_datasets["train"][0]["tokens"]
labels = [int(n) for n in raw_datasets["train"][0]["ner_tags"]]
#labels = raw_datasets["train"][0]["pos_tags"]
#labels = raw_datasets["train"][0]["chunk_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

Mujer de 43 años de edad , ama de casa , alérgica al contraste yodado y fumadora de 1 paquete diario de cigarrillos , histerectomizada y ooferectomizada derecha por endometriosis . 
 La paciente se encuentra en estudio por el Servicio de Hematología por leucocitosis aislada ; se le realizó ecografía abdominal hallando un riñón izquierdo desestructurado con imágenes quísticas o hidronefrosis y siendo remitida a nuestras consultas . 
 En el interrogatorio refiere un ingreso de 27 días de duración , cuando tenía 12 años , en otro hospital por dolor lumbar con hematuria sin conocer su diagnóstico . También refiere ocasionales dolores lumbares de tipo mecánico con astenia moderada . 
 Exploración física : sin hallazgos 
 Pruebas complementarias : hemograma con ligera leucocitosis sin desviación de fórmula leucocitaria siendo el resto normal , bioquímica y coagulación normales . 
 Renograma MAG-3-Tc-99 m : riñón derecho normal . Captación heterogénea en riñón izquierdo con dilatación calicil

## Loading mBERT as a pre-trained model

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "abs

In [ ]:
tokenizer.is_fast

True

In [ ]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

Token indices sequence length is longer than the specified maximum sequence length for this model (536 > 512). Running this sequence through the model will result in indexing errors


['[CLS]',
 'Mujer',
 'de',
 '43',
 'años',
 'de',
 'edad',
 ',',
 'ama',
 'de',
 'casa',
 ',',
 'al',
 '##ér',
 '##gica',
 'al',
 'contraste',
 'yo',
 '##dado',
 'y',
 'fu',
 '##mad',
 '##ora',
 'de',
 '1',
 'pa',
 '##quet',
 '##e',
 'diario',
 'de',
 'ci',
 '##garri',
 '##llos',
 ',',
 'his',
 '##tere',
 '##cto',
 '##mi',
 '##zada',
 'y',
 'o',
 '##of',
 '##ere',
 '##cto',
 '##mi',
 '##zada',
 'derecha',
 'por',
 'end',
 '##ome',
 '##tri',
 '##osis',
 '.',
 'La',
 'paciente',
 'se',
 'encuentra',
 'en',
 'estudio',
 'por',
 'el',
 'Servicio',
 'de',
 'He',
 '##mat',
 '##ología',
 'por',
 'le',
 '##uco',
 '##cito',
 '##sis',
 'ai',
 '##sla',
 '##da',
 ';',
 'se',
 'le',
 'realizó',
 'e',
 '##co',
 '##grafía',
 'ab',
 '##dom',
 '##inal',
 'hall',
 '##ando',
 'un',
 'ri',
 '##ñón',
 'izquierdo',
 'des',
 '##est',
 '##ru',
 '##ctura',
 '##do',
 'con',
 'imágenes',
 'qu',
 '##ísticas',
 'o',
 'hi',
 '##dron',
 '##ef',
 '##ros',
 '##is',
 'y',
 'siendo',
 're',
 '##mitida',
 'a',
 'nuestra'

In [ ]:
inputs.word_ids()

[None,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 11,
 11,
 12,
 13,
 14,
 14,
 15,
 16,
 16,
 16,
 17,
 18,
 19,
 19,
 19,
 20,
 21,
 22,
 22,
 22,
 23,
 24,
 24,
 24,
 24,
 24,
 25,
 26,
 26,
 26,
 26,
 26,
 26,
 27,
 28,
 29,
 29,
 29,
 29,
 30,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 42,
 42,
 43,
 44,
 44,
 44,
 44,
 45,
 45,
 45,
 46,
 47,
 48,
 49,
 50,
 50,
 50,
 51,
 51,
 51,
 52,
 52,
 53,
 54,
 54,
 55,
 56,
 56,
 56,
 56,
 56,
 57,
 58,
 59,
 59,
 60,
 61,
 61,
 61,
 61,
 61,
 62,
 63,
 64,
 64,
 65,
 66,
 66,
 67,
 67,
 68,
 70,
 71,
 72,
 72,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 92,
 93,
 94,
 94,
 94,
 95,
 96,
 97,
 98,
 98,
 99,
 100,
 101,
 102,
 102,
 102,
 103,
 103,
 104,
 104,
 104,
 105,
 106,
 107,
 107,
 107,
 108,
 109,
 109,
 110,
 110,
 111,
 113,
 113,
 113,
 114,
 115,
 116,
 117,
 117,
 117,
 119,
 119,
 119,
 120,
 120,
 121,
 122,
 122,
 122,
 123,
 124,
 124,
 

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    0,    0,  ...,    0,    0, -100],
        [-100,    0,    0,  ...,    0,    0, -100]])

In [ ]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 

In [ ]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_metric

metric = load_metric("seqeval")

In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'I',
 'I',
 'I',
 'O',
 'B',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'I',
 'I',
 'I',
 'I',
 'I',
 'O',
 'B',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'I',
 'I',
 'I',
 'I',
 'I',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O'

In [ ]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'_': {'f1': 1.0, 'number': 17, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0}

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
id2label

{'0': 'O', '1': 'B', '2': 'I'}

In [ ]:
label2id

{'B': '1', 'I': '2', 'O': '0'}

## Changing the head of prediction for Disease Mentions Identification under the BIO scheme

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(    
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
    num_labels = 3,
)

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B",
    "2": "I"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B": "1",
    "I": "2",
    "O": "0"
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_n

In [ ]:
model.config.num_labels

3

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    "NLP-CIC-WFU_DisTEMIST_fine_tuned_bert-base-multilingual-cased",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    num_train_epochs=7,
    weight_decay=0.01,
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


## Fine-tuning mBERT for Disease mentions identification

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 562
  Num Epochs = 7
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 497


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.168280,0.344828,0.461812,0.394837,0.933139
2,No log,0.150811,0.480986,0.531676,0.505062,0.947419
3,No log,0.149166,0.513579,0.503848,0.508667,0.948371
4,No log,0.158576,0.515303,0.548253,0.531268,0.950063
5,No log,0.182369,0.532330,0.565423,0.548378,0.950694
6,No log,0.189381,0.521022,0.579633,0.548767,0.951385
7,No log,0.204927,0.522951,0.566607,0.543905,0.952016


***** Running Evaluation *****
  Num examples = 188
  Batch size = 8
Saving model checkpoint to NER_EHR_Spanish_model_bert-base-multilingual-cased/checkpoint-71
Configuration saved in NER_EHR_Spanish_model_bert-base-multilingual-cased/checkpoint-71/config.json
Model weights saved in NER_EHR_Spanish_model_bert-base-multilingual-cased/checkpoint-71/pytorch_model.bin
tokenizer config file saved in NER_EHR_Spanish_model_bert-base-multilingual-cased/checkpoint-71/tokenizer_config.json
Special tokens file saved in NER_EHR_Spanish_model_bert-base-multilingual-cased/checkpoint-71/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 188
  Batch size = 8
Saving model checkpoint to NER_EHR_Spanish_model_bert-base-multilingual-cased/checkpoint-142
Configuration saved in NER_EHR_Spanish_model_bert-base-multilingual-cased/checkpoint-142/config.json
Model weights saved in NER_EHR_Spanish_model_bert-base-multilingual-cased/checkpoint-142/pytorch_model.bin
tokenizer config file saved

TrainOutput(global_step=497, training_loss=0.08985690571652571, metrics={'train_runtime': 537.2858, 'train_samples_per_second': 7.322, 'train_steps_per_second': 0.925, 'total_flos': 1027794567503484.0, 'train_loss': 0.08985690571652571, 'epoch': 7.0})

## Saving the fine-tuned model at Hugging Face (It requires previous authentication)

In [ ]:
trainer.push_to_hub(commit_message="Training complete")

## Analyzing predictions

In [ ]:
import numpy as np

predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

preds = np.argmax(predictions.predictions, axis=-1)


In [ ]:
i=0
print(raw_datasets["validation"][i]['tokens'])
for j in range(len(preds[i])):
  print(raw_datasets["validation"][i]['ner_tags'][j], "\t", preds[i][j])
print(' '.join(raw_datasets["validation"][i]['tokens']))

## Loading the model for inference

In [ ]:
from transformers import pipeline

#Replace this with your own checkpoint. If you have run all the previous cells successfully, the model should be available at your hugging face account with the name: NLP-CIC-WFU_DisTEMIST_fine_tuned_bert-base-multilingual-cased
model_checkpoint = "ajtamayoh/NER_EHR_Spanish_model_Mulitlingual_BERT"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)

In [ ]:
#pred = token_classifier("Mujer de 74 años que ingresó en el hospital por obnubilación y anuria tras presentar durante 5 días dolor abdominal y vómitos.")
pred = token_classifier("Entre sus antecedentes destacaba una DM tratada con metformina (850 mg/8 h) y glibenclamida.")
#pred = token_classifier("Anamnesis\nPresentamos el caso de un varón de 69 años de edad con antecedentes de ángor estable desde hace más de un año, remitido desde Atención Primaria por síntomas de tracto urinario inferior. El paciente presenta una puntuación de síntomas prostáticos (IPSS) de 9; refiere erección satisfactoria, con test de la disfunción eréctil (IIEF-5) de 23.\n\nExploración física\nComplexión corporal normal, con un índice de masa corporal de 20. Al tacto rectal se palpa una próstata sin hallazgos patológicos de interés.\n\nPruebas complementarias\n• A nivel analítico se evidencia un antígeno prostático específico (PSA) de 7,22 con cociente PSA libre/PSA total de 0,19.\n• En la ecografía transrectal se aprecia una próstata de 44 g.\n• Se realiza una biopsia transrectal ecodirigida de próstata, con resultado de adenocarcinoma de próstata Gleason 6 (3 + 3) en el lóbulo derecho.\n\nTratamiento y evolución\nTras ofrecer las distintas opciones de tratamiento, el paciente opta por prostatectomía radical robótica. Se realiza un abordaje intraperitoneal con movilización vesical para acceder al espacio de Retzius. Tras la liberación de la grasa periprostática y la apertura de la fascia endopélvica se advierte la presencia de arterias pudendas accesorias (APA) bilaterales. Ambas APA transcurren en íntima relación con la cápsula prostática, siendo necesaria una disección minuciosa para su conservación.\nTras identificar el cuello vesical, se procede a su sección para acceder al espacio retroprostático. Se disecan y liberan los conductos deferentes y vesículas seminales. A continuación se procede a una disección interfascial bilateral de bandeletas neurovasculares para su preservación. Tras la sección de los ligamentos puboprostáticos y del plexo de Santorini se secciona la uretra. Una vez preservadas ambas APA se procede a la anastomosis vesicouretral con sutura de monofilamento reabsorbible según la técnica de Van Velthoven. Por último, se procede al llenado de la vejiga para comprobar la estanqueidad de la sutura. Sangrado aproximado intraoperatorio de 300 ml, tiempo de consola 120 minutos.\nEn las revisiones ulteriores el paciente recupera de forma progresiva la función eréctil, consiguiendo en la última revisión al año un IIEF de 17. El valor del PSA en la última revisión se mantiene en 0,00.\n")
#pred = token_classifier("El segundo caso que se presenta es el de una lactante de 2 meses que acudió a urgencias por febrícula y un cuadro de urticaria aguda, aparentemente pruriginoso, de 4 días de evolución. Inicialmente afectaba la cara y las extremidades superiores, extendiéndose en pocas horas al tronco y las extremidades inferiores. Exantema urticariforme. No había afectación palmoplantar. Estas manifestaciones no se acompañaban de angioedema acral, labial ni lingual.\n\nComo antecedente epidemiológico destacable, la paciente convivía con 2 personas con COVID-19 demostrada, por lo que se realizó PCR a SARS-CoV-2 en aspirado nasofaríngeo, que fue positiva. Se pautó tratamiento sintomático vía oral con buena respuesta. La duración de la mayoría de las lesiones fue inferior a 24h, resolviéndose la clínica cutánea en 5 días, sin otras manifestaciones asociadas.")
#pred = token_classifier("Mujer de 74 años que ingresó en el hospital por obnubilación y anuria tras presentar durante 5 días dolor abdominal y vómitos. Entre sus antecedentes destacaba una DM tratada con metformina (850 mg/8 h) y glibenclamida. La presión arterial era de 105/60 mmHg, la frecuencia cardíaca de 155 latidos/minuto y la temperatura de 36,7º C. En la exploración destacaba deshidratación intensa, desorientación, respiración de Kussmaul, dolor abdominal con peristaltismo débil y ausencia de defensa abdominal. La tabla 2 muestra los principales datos analíticos, destacando además una hiperamilasemia de 2.605 U/l. Una tomografía computarizada abdominal descubrió un aumento del tamaño de la cabeza del páncreas sugerente de pancreatitis aguda. Tras suspender la administración de metformina, la paciente fue intubada orotraquealmente y conectada a un respirador mecánico. También recibió tratamiento a base de fluidos con suplementos de potasio, noradrenalina, bicarbonato, insulina, amiodarona, imipenen y furosemida. Al segundo día de ingreso, tras administrar 750 mEq de bicarbonato y 140 mEq de potasio, se normalizó el equilibrio ácido-básico (pH 7,41 y bicarbonato 20 mEq/l) e hidroelectrolítico (sodio 147 mEq/l, potasio 3,5 mEq/l) y la creatinina descendió a 5,5 mg/dl. Al tercer día de ingreso, por la presencia de deposiciones mucosas repetidas, se realizó una colonoscopia que puso de manifiesto una formación polipoidea sésil situada junto al esfínter anal y de 14 cm de longitud. La muestra endoscópica fue informada como adenoma velloso de recto. Al séptimo día de ingreso la enferma fue extubada, y dos días más tarde fue trasladada a planta para extirpación del adenoma")
#pred = token_classifier("Hombre de 38 años. Ingresó a la UCI (14/IX/ 2000), en postquirúrgico inmediato por heridas múltiples por arma de fuego en miembro superior derecho y abdomen. Hemicolectomía derecha con anastomosis termino-terminal y drenaje de hemoperitoneo (2000 cc.). Trece horas después, fue reintervenido por hipotensión y anemia, drenándose un hemoperitoneo de 3.000 cc. Bajo la sospecha de coagulopatía se realizó 48 horas de empaquetamiento y posteriormente malla de Velcro. El paciente presentó acidosis metabólica, insuficiencia renal aguda (el día 17, en remisión el 18) y coagulación intravascular diseminada.Fiebre el 17, 18 y 19. El 19 se llevó a un lavado abdominal (cierre fallido). Requirió ventilación mecánica desde el postquirúrgico hasta el 18 y se reinició el 19.Paraclínicos: Sep 17: Hb 7.8 gm/dl. Sep 18: leucocitos: 6.600, hipomagnesemia. Sep 19: hipofosfatemia (0.9 meq/lt). Sep 20: creatinina normal, hipofosfatemia.Medicación: Sep 18: Suspendida sedación (se inició en el postquirúrgico), recibe Ranitidinay Morfina. Septiembre 19: suspenden Morfina, reinician sedación bajo ventilación mecánica. Corrección de hipofosfatemia.\n\nDescripción y tratamiento de las alteraciones del comportamiento\n18 de septiembre: obedece órdenes sencillas. Presentó agitación leve, poca colaboración yconfusión. Se le diagnosticó un delirium para el cual se le formulo Haloperidol, la primeradosis se le suministró a las 17:00 horas 1 mg. I.V. y luego se repitió cada 8 horas.Septiembre 19: igual dosis de Haloperidol.Sep 20: fue desintubado con agitación motora severa y confuso, se aumentó la dosis deHaloperidol a 2 mg c/8 horas I.V. Al no haber control de la alteración se incrementó elHaloperidol a 3 mg cada 4 horas junto con Midazolam 2mg (5 bolos). Ante la no mejoríase le administró Fenergan 50mg I.M. e infusión de Haloperidol 2mg/cada hora. Ante lasituación se hizo IC a Psiquiatría de Enlace.El psiquiatra encontró que tenía una historia de consumo de alcohol semanal. Personalidadprevia: ansioso, íaceleradoî, sin antecedentes claros de otra enfermedad diferente a abusode alcohol. Los informantes eran poco confiables porque no vivían continuamente con él(exesposa e hija). En esa fecha los datos positivos del exámen mental fueron los siguientes: inquietud, con evidencia de alucinaciones visuales, desorientación parcial en persona, lugar, tiempo y enfermedad; inmovilizado.Se diagnóstica delirium y se comienza a descartar un sindrome de abstinencia a alcohol uotros psicotóxicos. Se continuó el tratamiento con Haloperidol 10 mg I.V. cada 20 minutos,hasta alcanzar un total de 80mg. Se solicitó tomografía cerebral informada normal; se hizocorrección de electrolitos (hiponatremia: 138); las pruebas hepáticas fueron normales. Comola agitación persistía, se incrementó el Haloperidol hasta 180 mg junto con 5 bolos de 2 mgde Midazolam. Como no se logró controlar la agitación, se le suministró infusión de PROPOFOL (anestésico endovenoso) 120 mg, logrando control del cuadro clínico.Septiembre 21: disnea con infiltrado alveolar. Hipomagnesemia. En el liquido peritoneal seencontró Serratia Marcenses y Stafilococo Aureus. Delirium mejor: orientado en todas lasáreas. Refiere consumo de una botella semanal de alcohol.Se continuó con 90 mg. I.V. díarios de Haloperidol (15 mg cada 4 horas), Loracepam 1mgcada 6 horas V.O. Se suspende ranitidina porque puede agravar el delirium.Septiembre 22: se disminuye Haloperidol 10 mg c/4 horas I.V. El cultivo del líquido peritoneal positivo con resistencia a Cefotaxime, se inicia Ciprofloxacina, Vancomicina. Teníainfiltrados alveolares, fiebre, derrame pleural. Exámen mental: orientado, describe el trauma, refiere dificultades para tolerar la frustración, se considera como diagnóstico un posible Trastorno de la Personalidad. Se suspende Haloperidol y se deja Lorazepam 1 mgcada 12 horas, vía oral.Septiembre 23: sale de la UCI a habitación corriente.Septiembre 25: informa que es consumidor habitual de alcohol y cocaína. Se autoriza salida de la institución")
#pred = token_classifier("Niña de 4 años, sin antecedentes de interés, que consulta por tumefacción cervical izquierda dolorosa y fiebre (38oC), de pocas horas de evolución. El día anterior, tras quejarse de dolor a nivel occipital, lo padres observaron una garrapata en cuero cabelludo que retiraron manualmente. Tres días antes, la niña había estado en una zona rural en contacto con animales.\n\nEn la exploración física, se observa sobre la zona de picadura de la garrapata, en región parietoccipital izquierda, una placa eritematosa de 1 cm de diámetro con escara central, tumefacción cervical ipsilateral dolorosa, sin exantemas, siendo el resto de exploración normal. Placa de 1 cm de diámetro, con escara necrótica central tras picadura de garrapata.\n\nSe realiza hemograma y bioquímica con resultado normal, sin elevación de reactantes de fase aguda, y ecografía cervical, que muestra adenopatías de características inflamatorias, de predominio en región laterocervical izquierda, no abcesificadas, siendo la mayor de 3,2 x 1,2 cm. Ecografía cervical. Adenopatía de 3,2 x 1,2 cm, de características inflamatorias y no abcesificada.\n\nEvolución\nSe sospechó un cuadro de linfadenopatía transmitida por garrapata o TIBOLA (de su acrónimo inglés Tick-Borne Lymphadenopathy). Se pautó tratamiento con doxiciclina oral (5 mg/kg/día cada 12 horas) en una pauta corta de 2 días. Paciente quedo afebril en las primeras 24 horas, con disminución progresiva de la adenitis cervical hasta quedar asintomáticas. Se realizó serología al diagnóstico y a las 6 semanas frente a Rickettsia conorii y Borrelia burdorferi (únicas serologías disponibles en nuestra área) con resultado negativo.\n\nDentro de las numerosas zoonosis cuyo vector es la garrapata, es importante conocer las manifestaciones de aquellas que son más frecuentes en nuestro país. Entre estas, destaca la linfadenopatía transmitida por garrapata o TIBOLA. Esta entidad se engloba dentro del grupo de enfermedades conocidas como Fiebres Manchadas ocasionadas por bacterias del genero Rickettsia (cocobacilos gram negativos intracelulares) y transmitidas por garrapatas. El agente principal aislado, en estos casos, es la Rickettsia slovaca, si bien, se han aislado otros patógenos, como Rickettsia raoultii, Rickettsia riojay otras bacterias, en pacientes con la misma clínica. Los vectores son las garrapatas del género Dermacentor (Dermacentor marginatusen los países mediterráneos, y Dermacentor reticularis en el Centro y Este de Europa). En los últimos años, TIBOLA, que también se conoce como DEBONEL (Dermacentor-borne necrosis erythema Lymphadenopathy) o SENLAT (Scalp eschar with neck lymphadenopathy after a tick bite), tiene una incidencia mayor que la fiebre botonosa mediterránea (FBM); rickettsiosis, con la que se establece el principal diagnóstico diferencial de TIBOLA.")
#pred = token_classifier("La madre del paciente neonatal es una mujer embarazada de 34 años que vive cerca del mercado mayorista de marisco de Huanan (a unos 1,2 km de distancia), en Wuhan. No ha visitado el mercado durante su embarazo y su familia no tiene casos confirmados ni presuntos de COVID-19, pero en la misma comunidad en la que vive se han diagnosticado más de 15 personas. Tiene antecedentes de hipotiroidismo de 4 años de evolución y se ha tratado con fármacos por vía oral; no tiene antecedentes de hipertensión, diabetes ni cardiopatías. Tuvo un aborto en 2016 a causa de alteraciones cromosómicas. Es alérgica a la penicilina y a las cefalosporinas de primera generación (positivo en pruebas cutáneas).\n\nA las 20:00 h del 1 de febrero de 2020, la mujer, de 40 semanas de gestación, presentó una pequeña hemorragia vaginal y dolor en la región abdominal inferior. Dos horas después, presentó fiebre (37,8 °C) y acudió al centro de asistencia maternoinfantil de Wuhan. Como tenía fiebre, fue derivada al consultorio de enfermedades infecciosas del hospital Tongji de Wuhan a la mañana siguiente. Una TAC torácica mostró opacidades de vidrio esmerilado en los lóbulos superior e inferior izquierdos, lo que indicaba la posibilidad de neumonía vírica.\n\nTAC torácica de la madre, obtenida el 2 de febrero de 2020, que muestra signos de infección en los lóbulos inferior y superior izquierdos e indica la posibilidad de neumonía vírica, con enfisema reducido en el lóbulo inferior derecho y una pequeña afectación en el lóbulo medio derecho.\n\nLos análisis de sangre mostraron linfocitopenia (0,97 × 109 células/L [normal: 1,1–3,2 × 109 células/L]), neutrofilia (9,97 × 109 células/L [normal: 1,8–6,3 × 109 células/L]) y concentraciones altas de proteína C-reactiva (11,5 mg/L [normal: < 1 mg/L]). Se decidió hospitalizarla por presunta neumonía vírica.\n\nAl ingreso, su temperatura era de 37,8 °C, la presión arterial de 131/89 mmHg, la frecuencia respiratoria de 20 r.p.m. y el pulso de 96 l.p.m. No presentaba tos ni expectoración. El pulso fetal era de 136 l.p.m. y su registro no mostró anomalías. Se realizó una cesárea de urgencia. Intraoperatoriamente se observó líquido amniótico con meconio. A las 8:45 la paciente dio a luz un niño con un peso de 3025 g. Los índices de Apgar a 1 y 5 minutos fueron de 8 y 9, respectivamente. El niño no regurgitó tras el parto. La piel era rojiza y el lloro, alto. La madre llevó una mascarilla N95 durante la intervención y, tras el alumbramiento, el niño no tuvo contacto con la madre. El niño se transfirió a la planta de neonatología 10 minutos después del parto para estrecha vigilancia; la madre se transfirió a la planta de enfermedades infecciosas para su aislamiento tras la intervención.\n\nMedia hora después del parto, el niño vomitó una vez tras ser alimentado con leche artificial, y se consideró que tenía disfagia. Tras un lavado gástrico, el niño pudo ser alimentado sin complicaciones. Los análisis de sangre del recién nacido mostraron linfocitopenia (2,43 × 109 células/L [normal: 3–8 × 109 células/L]), pruebas funcionales hepáticas alteradas (aspartato-aminotransferasa 143 U/L [normal: ≤  41 U/L]; bilirrubina total 33,0 μmol/L [normal: ≤ 26 μmol/L]; bilirrubina indirecta 26,0 μmol/L [normal: ≤ 16,8 μmol/L]) y alta concentración de creatina-cinasa (479 U/L [normal: ≤  41 U/L]). Se administró penicilina G (150.000 U una vez por día, bolo intravenoso) y vitamina K1 (1 mg una vez por día, i.v.) como profilaxis antibiótica y para evitar coagulopatías, respectivamente.\n\nLa madre permaneció en buen estado y afebril durante el periodo posoperatorio inmediato. No presentó tos ni ninguna otra molestia, como diarrea, náuseas o vómitos. Sus constantes vitales fueron estables, con una saturación de oxígeno del 99%. Se le administró un tratamiento antivírico, con 40 μg de interferón α1b humano recombinante atomizado por vía inhalatoria con 2 mL de solución de esterilización dos veces por día y ganciclovir (0,25 g cada 12 horas, i.v.). También se le administró abipenem (0,3 g cada 12 horas, i.v.) y moxifloxacina (0,4 g una vez por día, i.v.) para prevenir infecciones. La madre tuvo fiebre intermitente durante el primer día del posoperatorio, llegando a 38,3 °C; se le administró metilprednisolona (20 mg i.v.). Su frotis faríngeo para detección del SARS-CoV-2 resultó positivo ese mismo día. Inmediatamente, se obtuvo un frotis faríngeo del recién nacido (36 horas tras el parto) junto con leche de la madre. Recomendamos a la madre no amamantar al niño y extraerse leche para evitar mastitis.\n\nLa respuesta neurológica del recién nacido resultó aceptable durante el primer día después del parto y su saturación de oxígeno se mantuvo > 92% sin oxigenoterapia. Las pruebas analíticas del niño fueron negativas para Legionella pneumophila, Chlamydia pneumoniae, Mycoplasma pneumoniae, Rickettsia, adenovirus, virus respiratorio sincicial, virus de la gripe A virus de la gripe B y virus paragripal 1–3.\n\nEl 4 de febrero, el segundo día posoperatorio, las constantes vitales de la madre eran estables y se le administró prednisolona (40 mg una vez por día i.v.). El recién nacido se mantuvo sano y su gasometría mostró pH de 7,476↑, presión parcial de dióxido de carbono de 28,2 mm Hg↓, presión parcial de oxígeno de 116,0 mm Hg↑, bicarbonato de 20,6 mmol/L↓, exceso de base de 1,30 mmol/L y saturación de oxígeno periférico de 98,4%. Las pruebas para un conjunto de virus pediátricos resultaron negativas para citomegalovirus, virus de la rubéola, Toxoplasma gondii, virus del herpes común tipos 1 y 2, ecovirus, parvovirus B19, virus de Epstein-Barr, virus de Coxsackie A16, virus de Coxsackie B, virus del sarampión y virus de la varicela zóster. La TAC torácica del recién nacido mostró un engrosamiento de la textura de los pulmones sin anormalidades cardíacas. TAC torácica del recién nacido, obtenida el 4 de febrero de 2020, que muestra un engrosamiento de la textura de los pulmones sin anormalidades cardíacas. Se alimentó al niño con leche artificial, 25 mL cada 3 horas, y se supervisó estrechamente.\n\nEl 5 de febrero, las constantes vitales del recién nacido eran estables, con una saturación de oxígeno por encima del 90% y sin molestias como apnea o vómitos. El resultado del frotis faríngeo para detección del SARS-CoV-2 fue positivo a las 36 horas después del parto. Combinando todas las pruebas analíticas y un intercambio exhaustivo de ideas, diagnosticamos que el niño presentaba infección por SARS-CoV-2. Como el departamento neonatal del Hospital Tingji no dispone de las condiciones de aislamiento para el recién nacido, se transfirió al Hospital Maternoinfantil de Wuhan ese mismo día, para un mejor aislamiento. Tras hallar las pruebas de la infección neonatal, efectuamos pruebas de ácido nucleico del SARS-CoV-2 en la sangre del cordón umbilical y muestras de la placenta que habíamos conservado durante la intervención; los resultados fueron negativos. La muestra de la leche materna también fue negativa para SARS-CoV-2.\n\nHicimos un seguimiento del estado del recién nacido después de ser transferido al Hospital Maternoinfantil de Wuhan. Su estado era bueno y afebril, sin tos ni vómitos. Fue supervisado estrechamente en condiciones de aislamiento y no se le administró ningún tratamiento especial. Una TAC torácica el 6 de febrero mostró sombras nodulares de alta densidad bajo la pleura del segmento posterior del lóbulo superior del pulmón derecho. En una TAC torácica del 12 de febrero se observaron pequeños núcleos de sombras parcheadas en los lóbulos inferior y superior del pulmón derecho. En una TAC torácica del 17 de febrero se observaron unos pocos y pequeños núcleos de sombras parcheadas en los lóbulos inferior y superior del pulmón derecho, absorbidas en comparación con las anteriores. El 17 de febrero de 2020, las pruebas de ácido nucleico de frotis faríngeos y anales para detección del SARS-CoV-2 resultaron negativas. El recién nacido fue dado de alta el 18 de febrero de 2020.\n")
#pred = token_classifier("Mujer de 78 años, diagnosticada en 1978 de SSP por un síndrome seco (xerostomía y xeroftalmia), con biopsia de glándula salivar menor y pruebas de Schirmer y Rosa Bengala positivas.")
#pred = token_classifier("Paciente mujer de 84 años diagnosticada de artritis reumatoide, que ingresó de urgencia por dolor y distensión abdominal. Con el diagnóstico de íleo suboclusivo se instauró inicialmente tratamiento médico. Sin ningún antecedente urológico, comenzó a las pocas horas del sondaje uretral con hematuria, que se hizo muy intensa, evolucionando a la formación de un gran coágulo vesical que ocupaba toda la cavidad, observando además en la tomografía axial computarizada líquido y gas perivesical. Se procedió a intervención quirúrgica urgente, observando un tramo de íleon de 1,5 m de longitud hasta 10 cm de la válvula ileocecal, de aspecto violáceo edematoso, la cual se recupera adquiriendo aspecto, coloración y peristaltismo normales; presentaba una pequeña fisura en la pared vesical, por lo que se le practicó una cistotomía con la que se drena el gran coágulo, observando sangrado en sábana de la pared y gran friabilidad de la misma, tomando biopsias múltiples y realizando posteriormente ligadura de ambas hipogástricas. En el postoperatorio inmediato, tras un cuadro de anuria y alteración hemodinámica severa, llegó al exitus (09/05/2000). El diagnóstico anatomopatológico reveló que todas las biopsias vesicales que fueron practicadas tenían depósito amiloide (AA) vascular e intersticial.")
pred

In [ ]:
test_path = "/content/drive/MyDrive/Datasets/test_background/text_files/distemist_test_"

In [ ]:
i = 1
with open(test_path + str(i) + ".txt", "r", encoding="UTF-8") as ftest:
  pred = token_classifier(ftest.read())
pred

## Post-Processing

In [ ]:
def grouping_entities(pred):
  import re
  output = []
  for e in pred:
    if "##" not in e['word']:
      output.append(e)
    else:
      try:
        if e['start'] == (output[-1]['end']):
          output[-1]['word'] = output[-1]['word']+re.sub("##","",e['word'])
          output[-1]['end'] = e['end']
      except:
        pass
    
    try:
      if (e['entity_group'] == "B" or e['entity_group'] == "I") and (e['start'] == (output[-2]['end']+1)):
        output[-2]['word'] = output[-2]['word']+" "+e['word']
        output[-2]['end'] = e['end']
        output.pop(-1)
    except:
      pass
    
    try:
      if e['start'] == (output[-2]['end']):
        output[-2]['word'] = output[-2]['word']+e['word']
        output[-2]['end'] = e['end']
        output.pop(-1)
    except:
      pass

  return output


In [ ]:
grouping_entities(pred)

## Predictions on test datasets

In [ ]:
print("Processing...")
import re
f = open("/content/drive/MyDrive/distemist_subtrack1_test_predictions_NER_EHR_Spanish_model_Mulitlingual_BERT.tsv", "w", encoding="UTF-8")
#f.write("filename\tmark\tlabel\toff0\toff1\tspan\n")
f.write("filename\tlabel\toff0\toff1\tspan\n")
for i in range(1,3001):
  print(f"Text: {i}", end="\r")
  with open(test_path + str(i) + ".txt", "r", encoding="UTF-8") as ftest:
    hc = ftest.read()
    pred = token_classifier(hc)
    pred_grouped = grouping_entities(pred)
    t = 1
    for p in pred_grouped:

      off0 = int(p['start'])
      off1 = int(p['end'])
      span = hc[off0:off1]

      if span in [".", ",", ";", ":", '"', "-", "a", "de", "por", "in", "que", "da", "di", "se", "Las", "re", "sin"]:
        continue

      if "\n" in span:
        span = re.sub("\n"," ",span)

      if " - " in span:
        span = re.sub(" - ","-",span)
        off1 = off1-2

      if "( " in span:
        span = re.sub("\( ","(",span)
        off1 = off1-1

      if " )" in span:
        span = re.sub(" \)",")",span)
        off1 = off1-1

      if span.endswith(" y") :
        span = span[:-2]
        off1 = off1-2

      if span.endswith(" de") or span.endswith(" en"):
        span = span[:-3]
        off1 = off1-3

      if span.endswith(" por") or span.endswith(" con"):
        span = span[:-4]
        off1 = off1-4

      if span.endswith(".") or span.endswith(",") or span.endswith(";") or span.endswith(":") or span.endswith("–") or span.endswith("-"):
        span = span[:-1]
        off1 = off1-1

      if span.endswith(" .") or span.endswith(" ,") or span.endswith(" ;") or span.endswith(" :") or span.endswith(" –") or span.endswith(" -"):
        span = span[:-2]
        off1 = off1-2

      #f.write("distemist_test_"+str(i)+"\t"+"T"+str(t)+"\t"+"ENFERMEDAD"+"\t"+str(p['start'])+"\t"+str(p['end'])+"\t"+p['word']+"\n")
      #f.write("distemist_test_"+str(i)+"\t"+"T"+str(t)+"\t"+"ENFERMEDAD"+"\t"+str(off0)+"\t"+str(off1)+"\t"+span+"\n")
      f.write("distemist_test_"+str(i)+"\t"+"ENFERMEDAD"+"\t"+str(off0)+"\t"+str(off1)+"\t"+span+"\n")
      #print("distemist_test_"+str(i)+"\t"+"T"+str(t)+"\t"+"ENFERMEDAD"+"\t"+str(p['start'])+"\t"+str(p['end'])+"\t"+p['word'])
      t+=1
f.close()
print("Completo.")